In [4]:
import pandas as pd
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst




### 參考資料
https://pcjericks.github.io/py-gdalogr-cookbook/

[25.1507425088795,
 25.1508373135921,
 25.1510269230136,
 25.1511217277225,
 25.1514076742258,
 25.1513128695142,
 25.1507440412178,
 25.1506492364973,
 25.1506507686643,
 25.1507455733913,
 25.1515040111623,
 25.151598815878,
 25.1518847621051,
 25.1517899573867,
 25.1506523006666,
 25.1505574959317,
 25.1505590277627,
 25.1506538325041,
 25.1519810987518,
 25.1520759034743,
 25.1521722399734,
 25.1520774352456,
 25.150560559429,
 25.1504657546797,
 25.1504672861748,
 25.1505620909306,
 25.1520789668521,
 25.1521737715865,
 25.1520804982939,
 25.1519856935517,
 25.150848036547,
 25.1507532317884,
 25.1509443725026,
 25.1510391772653,
 25.1518924200722,
 25.1519872248223,
 25.1518939511715,
 25.1517991464136,
 25.1513251226055,
 25.1512303178401,
 25.1515162630408,
 25.151611067809,
 25.1518006773415,
 25.151895482106]

In [22]:
from pyproj import CRS

In [30]:
# 將csv轉成shp
import geopandas as gpd
from shapely.geometry import Point

flypath = pd.read_csv("C:/Users/USER/Desktop/飛行任務/dont_curved.csv") # test data
geometry = [Point(xy) for xy in zip(flypath["longitude"], flypath["latitude"])]
crs = 'EPSG:4326'
fp = gpd.GeoDataFrame(flypath, geometry=geometry, crs=crs)
fp.head() # GeoDataFrame

fp_3826 = fp.to_crs('EPSG:3826') # transform crs using geopandas method
# extract geometry by list, x and y
fp_3826_x = [point.x for point in fp_3826['geometry']]
fp_3826_y = [point.y for point in fp_3826['geometry']]


[305204.723173963, 305204.56735799526, 305204.25572431734, 305204.0999067072, 305219.38365638006, 305219.5394757103, 305220.47437999427, 305220.6301954638, 305236.3814027306, 305236.2255872863, 305234.9790437244, 305234.8232232571, 305250.10696706135, 305250.2627891519, 305252.1326112532, 305252.288426199, 305268.03963605745, 305267.8838210317, 305265.7023539588, 305265.5465307306, 305281.1419177822, 305281.29774157226, 305283.7908470693, 305283.9466614913, 305299.6978737391, 305299.54205923737, 305297.0489535701, 305296.89312980167, 305312.80016672146, 305312.9559899896, 305314.82582603476, 305314.9816421575, 305330.42122409696, 305330.2654068342, 305328.86302714853, 305328.70720448083, 305344.6142429745, 305344.7700651391, 305345.54916776775, 305345.704986652, 305360.98874535767, 305360.832924854, 305360.5212820999, 305360.36545985454]


In [6]:
# 創建點圖資
point = ogr.Geometry(ogr.wkbPoint)
point.AddPoint(301172,2770677)
print (point.ExportToWkt())

POINT (301172 2770677 0)


In [8]:
# 讀取raster
import sys
# this allows GDAL to throw Python Exceptions
gdal.UseExceptions()
path = "C:/Users/USER/Documents/GIS圖資/常用基本圖資/Raster/台北DEM/taipei dem3826.tif"

try:
    src_ds = gdal.Open(path)
except RuntimeError as e:
    print('Unable to open INPUT.tif')
    print(e)
    sys.exit(1)

try:
    srcband = src_ds.GetRasterBand(1)
except RuntimeError as e:
    # for example, try GetRasterBand(10)
    print('Band ( %i ) not found' % band_num)
    print(e)
    sys.exit(1)

In [9]:
# show the bonding box of rasterdata

geotransform = src_ds.GetGeoTransform()
# print(geotransform) (296110.0, 20.0, 0.0, 2789170.0, 0.0, -20.0) top left x, pixel size , rotation,  top left y, pixel size , rotation
width = src_ds.RasterXSize
height = src_ds.RasterYSize
min_x = geotransform[0]
max_x = geotransform[0] + (width * geotransform[1]) + (height * geotransform[2])
min_y = geotransform[3] + (width * geotransform[4]) + (height * geotransform[5])
max_y = geotransform[3]
box = [min_x,max_x,min_y,max_y]
print(box)
# elev = raster.data[301172,2770677]
# print(elev)

[296110.0, 317190.0, 2761549.9999, 2789169.9999]


In [10]:
raster_data = src_ds.ReadAsArray()
value = raster_data[1,2]
print(value)

-32767


In [44]:
type(src_ds)

osgeo.gdal.Dataset

In [11]:
longitude = 301172 
latitude = 2770677
raster_srs = osr.SpatialReference()
raster_srs.ImportFromWkt(src_ds.GetProjection())
print(raster_srs)


PROJCS["TWD97 / TM2 zone 121",
    GEOGCS["TWD97",
        DATUM["Taiwan_Datum_1997",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            AUTHORITY["EPSG","1026"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","3824"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",121],
    PARAMETER["scale_factor",0.9999],
    PARAMETER["false_easting",250000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","3826"]]


In [64]:
from affine import Affine
import numpy as np

# 座標轉換
lat = [25.0163750]
lon = [121.536791]
x_coords = [305283.946661491296254]
y_coords = [2782556.027021550107747]

# aff座標轉換有誤
# aff = Affine.from_gdal(*geotransform)
# x_coords, y_coords = aff*(np.array(lat),np.array(lon))
# print(x_coords, y_coords)


In [65]:
# https://gis.stackexchange.com/questions/225370/get-individual-pixel-values-on-a-raster-image-using-gdal-and-python?fbclid=IwAR1b0SvcY4L2QYzMK0lBAjvl_arGRCl0b7GX3jmZeUYwK_RyyWO2p-E2q5A#:~:text=from%20osgeo%20import%20gdal%20import%20sys%20%23%20coordinates,values%20value%20%3D%20data%20%5ByOffset%5D%20%5BxOffset%5D%20print%20value
band = src_ds.GetRasterBand(1)
xOrigin = geotransform[0]
yOrigin = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]

# loop through the coordinates
for i in range(len(lon)):
    x = x_coords[i]
    y = y_coords[i]
    # compute pixel offset
    xOffset = int((x - xOrigin) / pixelWidth)
    yOffset = int((y - yOrigin) / pixelHeight)
    # get individual pixel values
    data = band.ReadAsArray(xOffset, yOffset, 1, 1)
    value = data[0, 0]
    print(value)


305283.9466614913 2782556.02702155
458 330
409


In [31]:
# extract method

# I/O : Input 3826 list of x, y such that
# x = [305204.723173963022418,305220.630195463774726, ....]
# y = [2782586.360826591961086, 2782576.093745744321495, ...]
# raster path 

# import

from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst

import sys
import numpy as np



# data
path = "C:/Users/USER/Documents/GIS圖資/常用基本圖資/Raster/台北DEM/taipei dem3826.tif"
#x_coords = [305283.946661491296254,305220.6301954637]
#y_coords = [2782556.027021550107747,2782576.09374574]

x_coords = fp_3826_x
y_coords = fp_3826_y

# main

def extract(path:str,x_coords:list[float],y_coords:list[float])->list[float|int]:
    '''
    this methon can sample the values from DEM for given points.
    
    Parameters:
    -----
    path:str path of DEM dataset
    x_coords:list[float] x_coordinates of points
    y_coords:list[float] y_coordinates of points

    Output
    -----
    elev:list[float|int] list of elevation extract results
    

    '''
    # this allows GDAL to throw Python Exceptions
    gdal.UseExceptions()
    try:
        src_ds = gdal.Open(path)
    except RuntimeError as e:
        print('Unable to open INPUT.tif')
        print(e)
        sys.exit(1)

    try:
        band = src_ds.GetRasterBand(1)
    except RuntimeError as e:
        # for example, try GetRasterBand(10)
        print(e)
        sys.exit(1)

    geotransform = src_ds.GetGeoTransform() # (296110.0, 20.0, 0.0, 2789170.0, 0.0, -20.0) top left x, pixel size , rotation,  top left y, pixel size , rotation
    band = src_ds.GetRasterBand(1)
    xOrigin = geotransform[0]
    yOrigin = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]

    elev = [] # storage elevation result

    for i in range(len(x_coords)): # number of data
        x = x_coords[i]
        y = y_coords[i]
        # compute pixel offset
        xOffset = int((x - xOrigin) / pixelWidth)
        yOffset = int((y - yOrigin) / pixelHeight)
        # get individual pixel values
        data = band.ReadAsArray(xOffset, yOffset, 1, 1)
        value = data[0, 0]
        elev.append(value)

    return elev

result = extract(path,x_coords,y_coords)
print(result)

[412, 413, 414, 414, 416, 416, 411, 411, 410, 410, 417, 417, 418, 417, 409, 408, 408, 409, 418, 419, 419, 419, 409, 409, 414, 414, 420, 420, 420, 419, 415, 415, 417, 417, 419, 419, 420, 419, 418, 418, 419, 419, 420, 422]


In [38]:
base_hight = result[0]
delta_hight = result
for i in range(0,len(delta_hight)):
    delta_hight[i] = result[i]-base_hight
# print(delta_hight) # difference between orignal hight dem and way pts



[0, 1, 2, 2, 4, 4, -1, -1, -2, -2, 5, 5, 6, 5, -3, -4, -4, -3, 6, 7, 7, 7, -3, -3, 2, 2, 8, 8, 8, 7, 3, 3, 5, 5, 7, 7, 8, 7, 6, 6, 7, 7, 8, 10]


In [40]:
flypath["new_elev"] = flypath["altitude(m)"]+delta_hight
flypath["new_elev"]

0     50
1     51
2     52
3     52
4     54
5     54
6     49
7     49
8     48
9     48
10    55
11    55
12    56
13    55
14    47
15    46
16    46
17    47
18    56
19    57
20    57
21    57
22    47
23    47
24    52
25    52
26    58
27    58
28    58
29    57
30    53
31    53
32    55
33    55
34    57
35    57
36    58
37    57
38    56
39    56
40    57
41    57
42    58
43    60
Name: new_elev, dtype: int64